In [2]:
from os.path import join, exists
from os import listdir
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import anndata2ri
import rpy2
import numpy as np
%load_ext rpy2.ipython

import anndata2ri
anndata2ri.activate()

# import rpy2 and scran
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
r = robjects.r
importr("scran")


rpy2.robjects.packages.Package as a <module 'scran'>

In [4]:
adams = sc.read_h5ad('../data/adams/adams_input_scvi_Macrophage.h5ad')

In [3]:
df = []
ad_by_subject = {}

for f in listdir('data/adams/by_subject'):
    subject = f.split('.')[0]
    p = join('data/adams/by_subject', subject + '.h5ad')
    if not exists(p):
        continue

    outpath = join('data/adams/by_subject_scran/%s.h5ad' % subject)
    print(exists(outpath), outpath)
    if exists(outpath):
        continue
    
    print('reading %s' % p)
    ad = sc.read_h5ad(p)
    ad_by_subject[subject] = ad
    df.append([subject, ad.shape[0], ad.shape[1]])
    
df = pd.DataFrame(df, columns=['sample', 'n.cells', 'n.genes'])

False data/adams/by_subject_scran/235CO.h5ad
reading data/adams/by_subject/235CO.h5ad
True data/adams/by_subject_scran/123I.h5ad
False data/adams/by_subject_scran/098C.h5ad
reading data/adams/by_subject/098C.h5ad
False data/adams/by_subject_scran/065C.h5ad
reading data/adams/by_subject/065C.h5ad
True data/adams/by_subject_scran/465C.h5ad
True data/adams/by_subject_scran/238CO.h5ad
False data/adams/by_subject_scran/135I.h5ad
reading data/adams/by_subject/135I.h5ad
False data/adams/by_subject_scran/177I.h5ad
reading data/adams/by_subject/177I.h5ad
True data/adams/by_subject_scran/59I.h5ad
False data/adams/by_subject_scran/1372C.h5ad
reading data/adams/by_subject/1372C.h5ad
True data/adams/by_subject_scran/237CO.h5ad
True data/adams/by_subject_scran/021I.h5ad
False data/adams/by_subject_scran/063I.h5ad
reading data/adams/by_subject/063I.h5ad
False data/adams/by_subject_scran/210I.h5ad
reading data/adams/by_subject/210I.h5ad
False data/adams/by_subject_scran/483C.h5ad
reading data/adams/by

### Adams counts threshold filtering

In [30]:

# input_groups = ad_pp.obs['groups']
data_mat = ad[ad.obs.index.isin(set(ad.obs.head(100).index)),:].X.T.toarray()
# data_mat = adams.X.toarray()
data_mat.shape


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


(45947, 100)

In [ ]:
plot = False
for subject in sorted(ad_by_subject.keys(), key=lambda x: ad_by_subject[x].shape[0]):
    ad = ad_by_subject[subject]
    print(subject, ad.shape)
    outpath = join('data/adams/by_subject_scran/%s.h5ad' % subject)
    print(exists(outpath), outpath)
    if exists(outpath):
        continue
    
    
    # Quality control - calculate QC covariates
    ad.obs['n_counts'] = ad.X.sum(1)
    ad.obs['log_counts'] = np.log(ad.obs['n_counts'])
    ad.obs['n_genes'] = (ad.X > 0).sum(1)
    mt_gene_mask = [gene.startswith('MT-') for gene in ad.var['symbol']]
    ad.obs['mt_frac'] = ad.X[:, mt_gene_mask].toarray().sum(1) / ad.obs['n_counts']
    # Quality control - plot QC metrics
    
    if plot:
        #Sample quality plots
        t1 = sc.pl.violin(ad, 'n_counts', groupby='Subject_Identity', size=2, log=True, cut=0)
        t2 = sc.pl.violin(ad, 'mt_frac', groupby='Subject_Identity')
        #Data quality summary plots
        p1 = sc.pl.scatter(ad, 'n_counts', 'n_genes', color='mt_frac')
        p2 = sc.pl.scatter(ad[ad.obs['n_counts'] < 10000], 'n_counts', 'n_genes', color='mt_frac')
        #Thresholding decision: counts
        p3 = sns.distplot(ad.obs['n_counts'], kde=False)
        plt.show()
        p4 = sns.distplot(ad.obs['n_counts'][ad.obs['n_counts']<4000], kde=False, bins=60)
        plt.show()
        p5 = sns.distplot(ad.obs['n_counts'][ad.obs['n_counts']>10000], kde=False, bins=60)
        plt.show()
    
    print(ad.shape, ad[ad.X.sum(axis=1) > 1000,:].shape)
    ad = ad[ad.X.sum(axis=1) > 1000,:]
    ad = ad[ad.obs['mt_frac'] < 0.2,:]
    ad = ad[ad.obs['n_genes'] > 700,:]    
    
    print('normalizing...')
    ad_pp = ad.copy()
    sc.pp.normalize_per_cell(ad_pp, counts_per_cell_after=1e6)
    sc.pp.log1p(ad_pp)
    sc.pp.pca(ad_pp, n_comps=min(15, ad_pp.shape[0] - 1))
    sc.pp.neighbors(ad_pp)
    sc.tl.louvain(ad_pp, key_added='groups', resolution=0.5)
    
    input_groups = ad_pp.obs['groups']
    data_mat = ad.X.T.toarray()
    # data_mat = adams.X.toarray()

    import rpy2.robjects as robjects
    r = robjects.r
    from rpy2.robjects.packages import importr
    utils = importr("utils")

    data_mat = ad.X.T.toarray()

    print('calling scran...')
    size_factors = r['computeSumFactors'](data_mat, **{'min.mean': 0.1})

    print('done with scran...')
    #Delete morse_pp
    del ad_pp

    # Visualize the estimated size factors
    ad.obs['size_factors'] = size_factors
    if plot:
        sc.pl.scatter(ad, 'size_factors', 'n_counts')
        sc.pl.scatter(ad, 'size_factors', 'n_genes')
        sns.distplot(size_factors, bins=50, kde=False)
        plt.show()
        print('normalizing...')

    #Keep the count data in a counts layer
    ad.layers["counts"] = ad.X.copy()

    #Normalize adata 
    ad.X /= ad.obs['size_factors'].values[:,None]
    sc.pp.log1p(ad)

    ad.X = np.array(ad.X)
    # a the full data set in 'raw' as log-normalised data for statistical testing
    ad.raw = ad
    
    print('HVGs/clustering/saving...')
    sc.pp.highly_variable_genes(ad, n_top_genes=4000)
    # print('\n','Number of highly variable genes: {:d}'.format(np.sum(morse.var['highly_variable'])))

    if plot:
        sc.pl.highly_variable_genes(ad)

    # Calculate the visualizations
    sc.pp.pca(ad, n_comps=min(50, ad.shape[0] - 1), use_highly_variable=True, svd_solver='arpack')
    sc.pp.neighbors(ad)
    sc.tl.tsne(ad, n_jobs=12) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
    sc.tl.umap(ad)
    sc.tl.diffmap(ad)
    sc.tl.draw_graph(ad)

    if plot:
        sc.pl.pca_scatter(ad, color='n_counts')
        sc.pl.tsne(ad, color='n_counts')
        sc.pl.umap(ad, color='n_counts')
        sc.pl.diffmap(ad, color='n_counts', components=['1,2','1,3'])
        sc.pl.draw_graph(ad, color='n_counts')

    # Perform clustering - using highly variable genes
    sc.tl.louvain(ad, key_added='louvain_r1')
    sc.tl.louvain(ad, resolution=0.5, key_added='louvain_r0.5', random_state=10)

    #Visualize the clustering and how this is reflected by different technical covariates
    if plot:
        sc.pl.umap(ad, color=['louvain_r1', 'louvain_r0.5'], palette=sc.pl.palettes.default_102)
        sc.pl.umap(ad, color=['n_counts', 'log_counts', 'mt_frac'])

    print(outpath)
    ad.write(outpath, compression='lzf')
    print('saved...')
        

065C (1489, 45947)
False data/adams/by_subject_scran/065C.h5ad
(1489, 45947) (1489, 45947)
normalizing...


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


calling scran...


Trying to set attribute `.obs` of view, copying.


done with scran...
HVGs/clustering/saving...


...

data/adams/by_subject_scran/065C.h5ad


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


saved...
056CO (1652, 45947)
False data/adams/by_subject_scran/056CO.h5ad
(1652, 45947) (1652, 45947)
normalizing...


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


calling scran...
